<a href="https://colab.research.google.com/github/marina554/accounting-practice/blob/main/%E6%94%B9%E8%89%AF%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
journal_raw = [
    ("2025-11-01", "事務用品費", "現金", 5000),
    ("2025-11-02", "旅費交通費", "クレジットカード", 12000),
    ("2025-11-03", "売上", "銀行振込", 30000),
    ("2025-11-04", "通信費", "銀行振込", 8000),
    ("2025/11/05", "不明費用", "現金", -1000),  # 不正データ
]

journal_obj = Journal()

for rec in journal_raw:
    journal_obj.add_raw_entry(*rec)

# 一覧表示
print("【全仕訳一覧】")
journal_obj.show_all_info()

# DataFrame化
df = journal_obj.to_dataframe()
print("\n【DataFrame表示】")
print(df)

# 集計
print("\n【合計金額】")
print(journal_obj.total_amount())

print("\n【科目ごとの合計】")
print(journal_obj.total_by_category())


警告: 日付が不正です: 2025/11/05
【全仕訳一覧】
2025-11-01: 事務用品費, 5000円 [消耗品費 / 現金]
2025-11-02: 旅費交通費, 12000円 [旅費交通費 / 未払金]
2025-11-03: 売上, 30000円 [普通預金 / 売上高]
2025-11-04: 通信費, 8000円 [通信費 / 普通預金]

【DataFrame表示】
           日付   借方科目  貸方科目     金額     区分      支払方法
0  2025-11-01   消耗品費    現金   5000  事務用品費        現金
1  2025-11-02  旅費交通費   未払金  12000  旅費交通費  クレジットカード
2  2025-11-03   普通預金   売上高  30000     売上      銀行振込
3  2025-11-04    通信費  普通預金   8000    通信費      銀行振込

【合計金額】
55000

【科目ごとの合計】
区分
事務用品費     5000
売上       30000
旅費交通費    12000
通信費       8000
Name: 金額, dtype: int64


In [4]:
class Journal:
    def __init__(self):
        self.entries = []

    def add_entry(self, entry):
        if not isinstance(entry, JournalEntry):
            raise ValueError("JournalEntryオブジェクトを渡してください")
        self.entries.append(entry)

    def add_raw_entry(self, date, category, payment_method, amount):
        # 生データからJournalEntryを作り追加
        try:
            entry = JournalEntry(date, category, payment_method, amount)
            self.add_entry(entry)
        except ValueError as e:
            # エラーは警告として出力、処理は続行
            print(f"警告: {e}")

    def to_dataframe(self):
        return pd.DataFrame([e.to_dict() for e in self.entries])

    def total_amount(self):
        return sum(e.amount for e in self.entries)

    def total_by_category(self):
        df = self.to_dataframe()
        return df.groupby("区分")["金額"].sum()

    def show_all_info(self):
        for e in self.entries:
            print(e.info())


In [3]:
import pandas as pd
from datetime import datetime

class JournalEntry:
    def __init__(self, date, category, payment_method, amount):
        self.date = self._validate_date(date)
        self.category = category
        self.payment_method = payment_method
        self.amount = self._validate_amount(amount)
        self.debit, self.credit = self._map_accounts()

    # 日付バリデーション（不正なら例外）
    def _validate_date(self, date):
        try:
            # YYYY-MM-DD形式に統一
            return datetime.strptime(date, "%Y-%m-%d").date()
        except ValueError:
            raise ValueError(f"日付が不正です: {date}")

    # 金額バリデーション
    def _validate_amount(self, amount):
        if amount < 0:
            raise ValueError(f"金額が負です: {amount}")
        return amount

    # 借方・貸方マッピング
    def _map_accounts(self):
        debit_map = {
            "売上": "売上高",
            "仕入": "仕入高",
            "事務用品費": "消耗品費",
            "旅費交通費": "旅費交通費",
            "通信費": "通信費",
            "地代家賃": "地代家賃",
            "交際費": "交際費",
            "広告宣伝費": "広告宣伝費",
            "雑費": "雑費"
        }

        credit_map = {
            "現金": "現金",
            "銀行振込": "普通預金",
            "クレジットカード": "未払金",
            "掛け": "買掛金",
            "売掛": "売掛金"
        }

        debit = debit_map.get(self.category)
        credit = credit_map.get(self.payment_method)

        if debit is None or credit is None:
            raise ValueError(f"不正な科目または支払方法です: {self.category}, {self.payment_method}")

        if self.category == "売上":
            debit, credit = credit, debit

        return debit, credit

    def to_dict(self):
        return {
            "日付": self.date,
            "借方科目": self.debit,
            "貸方科目": self.credit,
            "金額": self.amount,
            "区分": self.category,
            "支払方法": self.payment_method
        }

    def info(self):
        return f"{self.date}: {self.category}, {self.amount}円 [{self.debit} / {self.credit}]"
